# The Transportation Problem           &emsp;   &emsp;      &emsp;       (*)

## Summary

Suppose we have a set of production **Plants** for a certain good, let's say milk, and a set of distant **Markets** that such Plants should supply in a given area.

Each plant has a milk production **Capacity** limit (cannot distribute more than a certain quantity) and each Market has a minimal **Demand** (minimal quantity that a market should get).

Assuming that we know the **transportation Cost** to bring 1 liter from each plant to every market, we want to setup the best distribution setup, that is, the setup that **minimizes the total transportation costs**.



![pic](images/OPT_Map_problem.png)
![pic2](images/OPT_Map_solution.png)


Problem             |  A solution
:-------------------------:|:-------------------------:
![pic3](images/OPT_Map_problem.png)  |  ![pic4](images/OPT_Map_solution.png)



## Problem Statement

The Transport Problem can be formulated mathematically using the following model.  

###  Sets
\begin{align}
&\text{set of Plants} &P & & = {p_1,p_2,p_3}  \\
&\text{set of Markets} &M & & = {m_1,m_2,m_3}
\end{align}
### Parameters
\begin{align}
&\text{Capacity of plant $p$ in liter} & C_p & &       &\forall p \in P  & & [\text{eg } & C_{p_1} &= 350 l &]\\
&\text{Demand at market $m$ in liter} & D_m & &      &\forall m \in M  & & [\text{eg } & D_{m_2} &= 600 l &]\\
&\text{Transport Cost in per liter}& Cost_{pm}& &   & \forall p \in P, m \in M & & [\text{eg } & Cost_{p_1m_2} &= 1.7  \text{ \$/l} &]\\
\end{align}
### Variables
\begin{align}
&\text{Shipment quantities} & x_{pm}    &\in  \mathrm{R\geq 0} & & \forall p \in P, m \in M \tag{V1}\\
\end{align}



### <span style="color:blue">Objective</span>
Minimize the total cost of the shipments: 
$$\min_{x} \sum_{p \in P} \sum_{m \in M} Cost_{pm} x_{pm} \tag{obj}$$
### <span style="color:blue">Constraints</span>

\begin{align}
&\text{Satisfy demand at market $p$}& &\sum_{p \in P} x_{pm} \geq D_{m} &\forall m \in M \tag{C1} & & [\text{eg } & & x_{p_1 m_2} + x_{p_2 m_2} + x_{p_3 m_2} &\geq 600 l &]\\
&\text{Observe supply limit at plant $p$}& &\sum_{m \in M} x_{pm} \leq C_{p} &\forall p \in P \tag{C2} & & [\text{eg }& & x_{p_1 m_1} + x_{p_1 m_2} + x_{p_1 m_3} &\leq 350 l &]\\
&\text{Non-negative transportation quantities}& &x_{pm} \geq 0 &\forall p \in P, \forall m \in M \tag{C3}
\end{align}


### Variables*
\begin{align}
& \text{Plant $p$ is active} &y_p & &  &\in \{0,1\} \tag{V2*}\\
\end{align}
### <span style="color:blue">Constraints*</span>
\begin{align}
&\text{Supply capacity if plant $p$ is active}& &\sum_{m \in J} x_{pm} \leq C_{p} * y_p & \forall p \in P \tag{C2*}\\
&\text{Total active Plants }& &\sum_{p \in P} y_p = N \tag{C4*}  & & [& y_{p_1} + y_{p_2} + y_{p_3} &=2  &  &]\\
\end{align}


# Implementation in PYOMO

In [1]:
from pyomo.environ import *


In [2]:
model = ConcreteModel()


In [3]:
#solver = SolverFactory("glpk")
solver = SolverFactory("gurobi")

#  Sets
\begin{align}
P & & \text{Plants} & & \textit{\{Seattle Plant, San-Diego Plant,Gotham City Plant\}}\\
M & & \text{Markets} & & \textit{\{New-York, Chicago, Topeka\}}
\end{align}

In [4]:
model.p = Set(initialize=['Seattle Plant','San-Diego Plant','Gotham City Plant'],
              doc='Plants')
model.m = Set(initialize=['New-York','Chicago', 'Topeka'], 
              doc='Markets')

#   Parameters


 * ${C}_p$  = 
    \begin{cases}
                 &\text{Seattle Plant}      &350 \\
                 &\text{San-Diego Plant}   &600 \\
                 &\text{Gotham City Plant} & 500\\
   \end{cases}
 * $D_m$  =
 \begin{cases}
                 &\text{New-York}     &350 \\
                 &\text{Chicago}   &600 \\
                 &\text{Topeka}    &  275
\end{cases}


In [5]:
model.C = Param(model.p, 
                 initialize={'Seattle Plant':350,'San-Diego Plant':600,'Gotham City Plant':500},
                 mutable=True, doc='Capacity of plant p in liters')
model.D = Param(model.m, 
                 initialize={'New-York':325,'Chicago':300,'Topeka':275},
                 mutable=True, doc='Demand at market m in liters')


* $Cost_{pm} =$

|   	 | \| |New-York |   Chicago|   Topeka|
|---	 | ---|---	  |---	    |---	|
|**Seattle**   | \| |   2.5|   1.7 |   1.8|
|**San-Diego** |\| |   2.5|   1.8|   1.4|
|**Gotham** |\| |   2.7|   3.3|   1.1|

In [6]:
ctab = {
    ('Seattle Plant',  'New-York') : 2.5,
    ('Seattle Plant',  'Chicago')  : 1.7,
    ('Seattle Plant',  'Topeka')   : 1.8,
    ('San-Diego Plant','New-York') : 2.5,
    ('San-Diego Plant','Chicago')  : 1.8,
    ('San-Diego Plant','Topeka')   : 1.4,
    ('Gotham City Plant','New-York') : 2.7,
    ('Gotham City Plant','Chicago')  : 3.3,
    ('Gotham City Plant','Topeka')   : 1.1
    }

model.Cost = Param(model.p, model.m, 
                initialize=ctab, 
                doc='Transport cost per liter',mutable=True)

#   Variables

*  $x_{pm} \in  \mathrm{R} $  <br/>
Non-negative transportation quantities $ x_{pm} \geq 0, \forall p \in P, \forall m \in M$
* $y_p  \in \{0,1\}$ 


In [7]:
model.x = Var(model.p, model.m, within=NonNegativeIntegers, doc='Shipment quantities in liter')
model.y = Var(model.p,within=Boolean,doc='Opened Plants')

# <span style="color:blue">Constraints</span>

 * Satisfy demand at market $m$ $ \sum_{m \in M} x_{pm} \geq D_{m}, \forall m \in M \tag{C2}$


In [8]:
def demand_rule(model, m):
    return sum(model.x[p,m] for p in model.p) >= model.D[m]  

model.demand = Constraint(model.m, rule=demand_rule, doc='Satisfy demand at market m')

* Supply capacity if plant $p$ is active $\sum_{m \in M} x_{pm} \leq C_{p} * y_p  \forall p \in P \tag{C1*}$



In [9]:
def supply_rule_if_active(model, p):
    return sum(model.x[p,m] for m in model.m) <= model.C[p] * model.y[p]

model.supply_if_active = Constraint(model.p, rule=supply_rule_if_active, doc='Observe supply limit at plant p if active')

* Total active plants 

$ \sum_{p \in P} y_p = 2 \tag{C4*}$

In [10]:
def tot_active(model,p):
    return sum(model.y[p] for p in model.p) == 2

model.tot_active = Constraint(model.p, rule=tot_active, doc='Total active plants')

# <span style="color:blue">Objective</span>

 * Minimize $  \sum_{p \in P} \sum_{m \in M} Cost_{pm} x_{pm} \tag{obj} $

In [11]:
def objective_rule(model):
    return sum( model.Cost[p,m] * model.x[p,m] for p in model.p for m in model.m)

model.objective = Objective(rule=objective_rule, sense=minimize, doc='Cost of transport')

In [12]:
def pprint_model(model):
    if model.solutions.solutions:
        print(model.x.display())
        print(model.y.display())
        print('Cost %.0f $' % model.objective())
    else:
        print('No Solution')

# Results
### Transport Problem

In [13]:
model_a = model.clone()
model_a.name = 'Original'

In [14]:
results_a = solver.solve(model_a,tee=True)


--------------------------------------------
--------------------------------------------

Using license file C:\Users\niccolomoretti\gurobi.lic
Read LP format model from file C:\Users\NICCOL~1\AppData\Local\Temp\tmpdb7ygldv.pyomo.lp
Reading time = 0.01 seconds
x13: 10 rows, 13 columns, 31 nonzeros
Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (win64)
Optimize a model with 10 rows, 13 columns and 31 nonzeros
Model fingerprint: 0x7e1295bf
Variable types: 1 continuous, 12 integer (3 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+02]
  Objective range  [1e+00, 3e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
Found heuristic solution: objective 2187.5000000
Presolve removed 3 rows and 1 columns
Presolve time: 0.01s
Presolved: 7 rows, 12 columns, 24 nonzeros
Variable types: 0 continuous, 12 integer (3 binary)

Root relaxation: objective 1.625000e+03, 7 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Wor

In [15]:
pprint_model(model_a)

x : Shipment quantities in liter
    Size=9, Index=x_index
    Key                               : Lower : Value : Upper : Fixed : Stale : Domain
     ('Gotham City Plant', 'Chicago') :     0 :  -0.0 :  None : False : False : NonNegativeIntegers
    ('Gotham City Plant', 'New-York') :     0 :  25.0 :  None : False : False : NonNegativeIntegers
      ('Gotham City Plant', 'Topeka') :     0 : 275.0 :  None : False : False : NonNegativeIntegers
       ('San-Diego Plant', 'Chicago') :     0 : 300.0 :  None : False : False : NonNegativeIntegers
      ('San-Diego Plant', 'New-York') :     0 : 300.0 :  None : False : False : NonNegativeIntegers
        ('San-Diego Plant', 'Topeka') :     0 :  -0.0 :  None : False : False : NonNegativeIntegers
         ('Seattle Plant', 'Chicago') :     0 :   0.0 :  None : False : False : NonNegativeIntegers
        ('Seattle Plant', 'New-York') :     0 :  -0.0 :  None : False : False : NonNegativeIntegers
          ('Seattle Plant', 'Topeka') :     0 :  -0.0 

### All active Plants

In [16]:
model_b = model.clone()
model_b.name = 'All Active Plants'

In [17]:
model_b.tot_active.deactivate()

In [18]:
results_b = solver.solve(model_b,tee=True)


--------------------------------------------
--------------------------------------------

Using license file C:\Users\niccolomoretti\gurobi.lic
Read LP format model from file C:\Users\NICCOL~1\AppData\Local\Temp\tmp725zmox_.pyomo.lp
Reading time = 0.00 seconds
x13: 7 rows, 13 columns, 22 nonzeros
Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (win64)
Optimize a model with 7 rows, 13 columns and 22 nonzeros
Model fingerprint: 0x19480e74
Variable types: 1 continuous, 12 integer (3 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+02]
  Objective range  [1e+00, 3e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
Found heuristic solution: objective 1715.0000000
Presolve removed 1 rows and 4 columns
Presolve time: 0.00s
Presolved: 6 rows, 9 columns, 18 nonzeros
Variable types: 0 continuous, 9 integer (0 binary)

Root relaxation: objective 1.625000e+03, 3 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 E

In [19]:
pprint_model(model_b)

x : Shipment quantities in liter
    Size=9, Index=x_index
    Key                               : Lower : Value : Upper : Fixed : Stale : Domain
     ('Gotham City Plant', 'Chicago') :     0 :  -0.0 :  None : False : False : NonNegativeIntegers
    ('Gotham City Plant', 'New-York') :     0 :  -0.0 :  None : False : False : NonNegativeIntegers
      ('Gotham City Plant', 'Topeka') :     0 : 275.0 :  None : False : False : NonNegativeIntegers
       ('San-Diego Plant', 'Chicago') :     0 :  -0.0 :  None : False : False : NonNegativeIntegers
      ('San-Diego Plant', 'New-York') :     0 : 325.0 :  None : False : False : NonNegativeIntegers
        ('San-Diego Plant', 'Topeka') :     0 :  -0.0 :  None : False : False : NonNegativeIntegers
         ('Seattle Plant', 'Chicago') :     0 : 300.0 :  None : False : False : NonNegativeIntegers
        ('Seattle Plant', 'New-York') :     0 :  -0.0 :  None : False : False : NonNegativeIntegers
          ('Seattle Plant', 'Topeka') :     0 :  -0.0 

In [20]:
print('Objective function:')
print('Original Model:                           %.0f$' % model_a.objective())
print('No-Constraint Model:                      %.0f$' % model_b.objective())


Objective function:
Original Model:                           1660$
No-Constraint Model:                      1625$


### Unfeasible Problem

In [21]:
model_c = model.clone()
model_c.name = 'No solution'

In [22]:
model_c.D['Topeka'] = 10000

In [23]:
results_c = solver.solve(model_c,tee=True)


--------------------------------------------
--------------------------------------------

Using license file C:\Users\niccolomoretti\gurobi.lic
Read LP format model from file C:\Users\NICCOL~1\AppData\Local\Temp\tmptkp9mwms.pyomo.lp
Reading time = 0.00 seconds
x13: 10 rows, 13 columns, 31 nonzeros
Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (win64)
Optimize a model with 10 rows, 13 columns and 31 nonzeros
Model fingerprint: 0x907accc2
Variable types: 1 continuous, 12 integer (3 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+02]
  Objective range  [1e+00, 3e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+04]
Presolve removed 1 rows and 1 columns
Presolve time: 0.00s

Explored 0 nodes (0 simplex iterations) in 0.00 seconds
Thread count was 1 (of 4 available processors)

Solution count 0

Model is infeasible
Best objective -, best bound -, gap -
    solution;
        message from solver=Model was proven to be infeasible.


In [24]:
pprint_model(model_c)

No Solution


## (*) Example taken from https://github.com/Pyomo/PyomoGallery and adapted